In [2]:
import cv2
import os
import numpy as np
from keras.preprocessing import image
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPool2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

In [53]:
from keras.preprocessing import image

In [20]:
import os
import cv2

def gen_dataset(enrolment):
    # Create a folder with the enrolment name if it doesn't exist
    student_folder = os.path.join("Students", enrolment)
    if not os.path.exists(student_folder):
        os.makedirs(student_folder)
    else:
        # Remove existing files in the folder if it already exists
        existing_files = os.listdir(student_folder)
        for existing_file in existing_files:
            file_path = os.path.join(student_folder, existing_file)
            os.remove(file_path)

    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.2, 5)
        cropped_faces = []
        for (x, y, w, h) in faces:
            cropped_face = img[y: y+h, x:x+w]
            cropped_faces.append(cropped_face)
        return cropped_faces
    
    cap = cv2.VideoCapture(0)  # Change the camera index if needed
    img_id = 0

    while True:
        ret, frame = cap.read()
        faces = face_cropped(frame)
        for face in faces:
            img_id += 1
            face = cv2.resize(face, (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_path = os.path.join(student_folder, f"{enrolment}.{img_id}.jpg")
            # cv2.putText(face, str(img_id), (40, 40), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)
            cv2.imshow("Cropped_Face", face)
            cv2.imwrite(file_path, face)

        if cv2.waitKey(1) == 15 or int(img_id) == 500:
            break

    cap.release()
    cv2.destroyAllWindows()
    print("Dataset Creation Completed")


In [83]:
gen_dataset("E21CSEU0130")

Dataset Creation Completed


In [74]:

# Specifying the folder where images are present
TrainingImagePath='Students'
 

# Understand more about ImageDataGenerator at below link
# https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
 
# Defining pre-processing transformations on raw images of training data
# These hyper parameters helps to generate slightly twisted versions
# of the original image, which leads to a better model, since it learns
# on the good and bad mix of images
train_datagen = ImageDataGenerator(
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)
 
# Defining pre-processing transformations on raw images of testing data
# No transformations are done on the testing images
test_datagen = ImageDataGenerator()
 
# Generating the Training Data
training_set = train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')
 
 
# Generating the Testing Data
test_set = test_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')
 
# Printing class labels for each face
test_set.class_indices

Found 900 images belonging to 3 classes.


Found 900 images belonging to 3 classes.


{'E21CSEU0106': 0, 'E21CSEU0130': 1, 'E21CSEU0143': 2}

In [75]:
# class_indices have the numeric tag for each face
TrainClasses=training_set.class_indices
 
# Storing the face and the numeric tag for future reference
ResultMap={}
for faceValue,faceName in zip(TrainClasses.values(),TrainClasses.keys()):
    ResultMap[faceValue]=faceName
 
# Saving the face map for future reference
import pickle
with open("ResultsMap.pkl", 'wb') as fileWriteStream:
    pickle.dump(ResultMap, fileWriteStream)
 
# The model will give answer as a numeric tag
# This mapping will help to get the corresponding face name for it
print("Mapping of Face and its ID",ResultMap)
 
# The number of neurons for the output layer is equal to the number of faces
OutputNeurons=len(ResultMap)
print('\n The Number of output neurons: ', OutputNeurons)

Mapping of Face and its ID {0: 'E21CSEU0106', 1: 'E21CSEU0130', 2: 'E21CSEU0143'}

 The Number of output neurons:  3


In [76]:


# Initializing the Convolutional Neural Network
classifier = Sequential()

# Adding the first layer of CNN
classifier.add(Convolution2D(32, kernel_size=(5, 5), strides=(1, 1), input_shape=(64, 64, 1), activation='relu'))

# MAX Pooling
classifier.add(MaxPool2D(pool_size=(2, 2)))

# Additional Layer of Convolution for better accuracy
classifier.add(Convolution2D(64, kernel_size=(5, 5), strides=(1, 1), activation='relu'))
classifier.add(MaxPool2D(pool_size=(2, 2)))

# Flattening
classifier.add(Flatten())

# Fully Connected Neural Network
classifier.add(Dense(64, activation='relu'))
classifier.add(Dense(OutputNeurons, activation='softmax'))

# Compiling the CNN
classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

# Data Augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# Provide the path to your dataset (make sure it has subfolders for each class)
train_set = train_datagen.flow_from_directory('Students',
                                              target_size=(64, 64),
                                              color_mode='grayscale',
                                              batch_size=32,
                                              class_mode='categorical')

test_set = test_datagen.flow_from_directory('Students',
                                            target_size=(64, 64),
                                            color_mode='grayscale',
                                            batch_size=32,
                                            class_mode='categorical')

classifier.fit_generator(train_set,
                         steps_per_epoch=len(train_set),
                         epochs=10,
                         validation_data=test_set,
                         validation_steps=len(test_set))

Found 900 images belonging to 3 classes.
Found 900 images belonging to 3 classes.


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_37912\2205836426.py:45: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(train_set,


Epoch 1/10
29/29 [==============================] - 9s 289ms/step - loss: 0.9419 - accuracy: 0.6033 - val_loss: 0.4370 - val_accuracy: 0.9444
Epoch 2/10
29/29 [==============================] - 4s 130ms/step - loss: 0.1992 - accuracy: 0.9356 - val_loss: 0.0206 - val_accuracy: 0.9978
Epoch 3/10
29/29 [==============================] - 4s 132ms/step - loss: 0.0545 - accuracy: 0.9833 - val_loss: 0.0072 - val_accuracy: 1.0000
Epoch 4/10
29/29 [==============================] - 4s 130ms/step - loss: 0.0125 - accuracy: 0.9978 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 5/10
29/29 [==============================] - 4s 131ms/step - loss: 0.0096 - accuracy: 0.9978 - val_loss: 0.0096 - val_accuracy: 1.0000
Epoch 6/10
29/29 [==============================] - 4s 141ms/step - loss: 0.0173 - accuracy: 0.9956 - val_loss: 0.0030 - val_accuracy: 0.9978
Epoch 7/10
29/29 [==============================] - 4s 137ms/step - loss: 0.0130 - accuracy: 0.9944 - val_loss: 0.0015 - val_accuracy: 1.0000
Epoch 

In [78]:
ImagePath = '../Attendify/Students/E21CSEU0130/E21CSEU0130.108.jpg'
test_image = image.load_img(ImagePath, target_size=(64, 64), color_mode='grayscale')
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

# Make a prediction
result = classifier.predict(test_image, verbose=0)
# Assuming you have a ResultMap defined somewhere, replace it with your own logic

print(ResultMap[np.argmax(result)])


E21CSEU0106


## Saving the model: 

also creating it and training it from scratch

In [17]:
from keras.models import Sequential, load_model
from keras.layers import Convolution2D, MaxPool2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
import pickle


# Data Augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# Provide the path to your dataset (make sure it has subfolders for each class)
train_set = train_datagen.flow_from_directory('Students',
                                              target_size=(64, 64),
                                              color_mode='grayscale',
                                              batch_size=32,
                                              class_mode='categorical')

test_set = test_datagen.flow_from_directory('Students',
                                            target_size=(64, 64),
                                            color_mode='grayscale',
                                            batch_size=32,
                                            class_mode='categorical')

TrainClasses=train_set.class_indices

ResultMap={}
for faceValue,faceName in zip(TrainClasses.values(),TrainClasses.keys()):
    ResultMap[faceValue]=faceName
 
# Saving the face map for future reference
with open("StudentsMap.pkl", 'wb') as fileWriteStream:
    pickle.dump(ResultMap, fileWriteStream)
 
# The model will give answer as a numeric tag
# This mapping will help to get the corresponding face name for it
 
# The number of neurons for the output layer is equal to the number of faces
OutputNeurons=len(ResultMap)

# Initializing the Convolutional Neural Network
classifier = Sequential()

# Adding the first layer of CNN
classifier.add(Convolution2D(32, kernel_size=(5, 5), strides=(1, 1), input_shape=(64, 64, 1), activation='relu'))

# MAX Pooling
classifier.add(MaxPool2D(pool_size=(2, 2)))

# Additional Layer of Convolution for better accuracy
classifier.add(Convolution2D(64, kernel_size=(5, 5), strides=(1, 1), activation='relu'))
classifier.add(MaxPool2D(pool_size=(2, 2)))

# Flattening
classifier.add(Flatten())

# Fully Connected Neural Network
classifier.add(Dense(64, activation='relu'))
classifier.add(Dense(OutputNeurons, activation='softmax'))

# Compiling the CNN
classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])


classifier.fit_generator(train_set,
                         steps_per_epoch=len(train_set),
                         epochs=10,
                         validation_data=test_set,
                         validation_steps=len(test_set))

# Save the trained model
classifier.save('RecognitionModel.h5')

Found 898 images belonging to 3 classes.


Found 898 images belonging to 3 classes.
Epoch 1/10


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19200\2780421680.py:68: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(train_set,


29/29 [==============================] - 6s 165ms/step - loss: 0.8655 - accuracy: 0.5568 - val_loss: 0.4509 - val_accuracy: 0.7695
Epoch 2/10
29/29 [==============================] - 4s 134ms/step - loss: 0.2433 - accuracy: 0.9220 - val_loss: 0.0495 - val_accuracy: 0.9989
Epoch 3/10
29/29 [==============================] - 4s 131ms/step - loss: 0.0909 - accuracy: 0.9677 - val_loss: 0.0227 - val_accuracy: 1.0000
Epoch 4/10
29/29 [==============================] - 4s 139ms/step - loss: 0.0455 - accuracy: 0.9844 - val_loss: 0.0101 - val_accuracy: 1.0000
Epoch 5/10
29/29 [==============================] - 4s 136ms/step - loss: 0.0251 - accuracy: 0.9922 - val_loss: 0.0041 - val_accuracy: 1.0000
Epoch 6/10
29/29 [==============================] - 4s 131ms/step - loss: 0.0245 - accuracy: 0.9955 - val_loss: 0.0025 - val_accuracy: 1.0000
Epoch 7/10
29/29 [==============================] - 4s 137ms/step - loss: 0.0117 - accuracy: 0.9978 - val_loss: 0.0032 - val_accuracy: 1.0000
Epoch 8/10
29/29 

In [25]:
from keras.models import load_model

In [63]:
saved_model_path = '../RecognitionModel.h5'
loaded_model = load_model(saved_model_path)

## Prediction

In [64]:
ImagePath = 'WIN_20231121_03_23_13_Pro.jpg'
test_image = image.load_img(ImagePath, target_size=(64, 64), color_mode='grayscale')
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

result = loaded_model.predict(test_image, verbose=0)
with open('../StudentsMap.pkl', 'rb') as file:
    # Load the object from the file
    loaded_object = pickle.load(file)
print(loaded_object[np.argmax(result)])

E21CSEU0143


In [49]:
resized_image = cv2.resize(all_faces_list[0][0], (64, 64))
gray_image = resized_image

# Expand dimensions to make it compatible with the model input shape
test_image = np.expand_dims(gray_image, axis=0)
test_image = np.expand_dims(test_image, axis=-1)  # Add channel dimension for grayscale

# Now, you can use the loaded model for prediction
result = loaded_model.predict(test_image, verbose=0)

with open('StudentsMap.pkl', 'rb') as file:
    # Load the object from the file
    loaded_object = pickle.load(file)

# Assuming loaded_object is a list or array mapping class indices to labels
predicted_label = loaded_object[np.argmax(result)]
print(predicted_label)

E21CSEU0106


In [46]:
resized_image = cv2.resize(all_faces_list[0][0], (64, 64))
gray_image = (resized_image)

In [47]:
gray_image

array([[180, 180, 180, ...,  27,  26,  24],
       [180, 180, 180, ...,  27,  26,  24],
       [180, 180, 180, ...,  27,  26,  24],
       ...,
       [ 23,  24,  25, ...,  20,  20,  22],
       [ 23,  25,  25, ...,  20,  20,  22],
       [ 24,  24,  24, ...,  20,  20,  22]], dtype=uint8)

## Testing 

In [4]:
import pickle
with open('../StudentsMap.pkl', 'rb') as file:
    # Load the object from the file
    loaded_object = pickle.load(file)
loaded_object

{0: 'E21CSEU0106',
 1: 'E21CSEU0130',
 2: 'E21CSEU0143',
 3: 'E21CSEU0149',
 4: 'E21CSEU0152'}

In [5]:
saved_model_path = '../RecognitionModel.h5'
loaded_model = load_model(saved_model_path)

In [6]:
camera = cv2.VideoCapture(0)  
while True:
 
    ret, frame = camera.read()

    width, height = 64, 64  # Set your desired width and height
    resized_frame = cv2.resize(frame, (width, height))

    gray_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)

    # Expand dimensions and add a channel dimension
    test_image = np.expand_dims(gray_frame, axis=0)
    test_image = np.expand_dims(test_image, axis=3)
    result = loaded_model.predict(test_image)
    label = loaded_object[np.argmax(result)]

    # Display the result on the frame
    cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Live Camera Feed', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()

cv2.destroyAllWindows()

1/1 [==============================] - 0s 28ms/step


## For retraining the loaded model

cannot be used, as we cannot update here the number of neurons

In [84]:
# Compile the model with the desired optimizer, loss, and metrics
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

# Data Augmentation for training the model
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Provide the path to your new training dataset
new_train_set = train_datagen.flow_from_directory('Students',
                                                 target_size=(64, 64),
                                                 color_mode='grayscale',
                                                 batch_size=32,
                                                 class_mode='categorical')

TrainClasses=new_train_set.class_indices
 
ResultMap={}
for faceValue,faceName in zip(TrainClasses.values(),TrainClasses.keys()):
    ResultMap[faceValue]=faceName
 
# Saving the face map for future reference
with open("StudentsMap.pkl", 'wb') as fileWriteStream:
    pickle.dump(ResultMap, fileWriteStream)
 
OutputNeurons=len(ResultMap)

classifier.add(Dense(OutputNeurons, activation='softmax'))

# Retrain the model with the new training data
loaded_model.fit_generator(new_train_set,
                            steps_per_epoch=len(new_train_set),
                            epochs=10)

# Save the retrained model
loaded_model.save('RecognitionModel.h5')

Found 900 images belonging to 3 classes.
Epoch 1/10


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_37912\1217135241.py:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  loaded_model.fit_generator(new_train_set,


29/29 [==============================] - 5s 157ms/step - loss: 0.1547 - accuracy: 0.9433
Epoch 2/10
29/29 [==============================] - 3s 103ms/step - loss: 0.0074 - accuracy: 1.0000
Epoch 3/10
29/29 [==============================] - 3s 105ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 4/10
29/29 [==============================] - 3s 104ms/step - loss: 0.0042 - accuracy: 0.9989
Epoch 5/10
29/29 [==============================] - 3s 110ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 6/10
29/29 [==============================] - 3s 111ms/step - loss: 8.6377e-04 - accuracy: 1.0000
Epoch 7/10
29/29 [==============================] - 3s 104ms/step - loss: 5.6720e-04 - accuracy: 1.0000
Epoch 8/10
29/29 [==============================] - 3s 103ms/step - loss: 4.3957e-04 - accuracy: 1.0000
Epoch 9/10
29/29 [==============================] - 3s 102ms/step - loss: 4.4790e-04 - accuracy: 1.0000
Epoch 10/10
29/29 [==============================] - 3s 104ms/step - loss: 9.8599e-04 - accurac

c:\Users\Lenovo\OneDrive\Desktop\Attendify\attendifyenv\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Extract Faces from students present image

In [22]:
import cv2
import os

# Load the face classifier
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def face_cropped(img):

    student_folder = os.path.join("StudentPresent")
    if not os.path.exists(student_folder):
        os.makedirs(student_folder)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the image
    faces = face_classifier.detectMultiScale(gray, 1.2, 5)

    # Initialize img_id
    img_id = 0

    # Iterate through detected faces
    for (x, y, w, h) in faces:
        # Crop the face from the original image
        cropped_face = img[y: y+h, x:x+w]

        # Resize the face to a common size (e.g., 200x200)
        cropped_face = cv2.resize(cropped_face, (200, 200))

        # Convert the face to grayscale
        cropped_face = cv2.cvtColor(cropped_face, cv2.COLOR_BGR2GRAY)

        # Save the face to a file
        file_path = os.path.join(student_folder, f"Student_{img_id}.jpg")
        cv2.imwrite(file_path, cropped_face)

        # Increment img_id for the next face
        img_id += 1

image_path = 'IMG_3877.HEIC.jpg'
img = cv2.imread(image_path)
face_cropped(img)


In [3]:
import requests
import cv2
import numpy as np
import imutils

# Getting Image and store it
def get_image(name):
    url = "http://10.12.60.98:8080//shot.jpg"
    img_resp = requests.get(url)
    img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
    img = cv2.imdecode(img_arr, -1)
    img = imutils.resize(img, width=1000, height=1800)

    # Storing the image in Caputres Folder as Android_cam.jpg
    cv2.imwrite(f"{name}.jpg", img)

In [5]:
import requests
import cv2
import numpy as np
import imutils
import time

def capture_images(url, interval_seconds=5, num_images=10):
    image_list = []

    for _ in range(num_images):
        img_resp = requests.get(url)
        img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
        img = cv2.imdecode(img_arr, -1)
        img = imutils.resize(img, width=1000, height=1800)

        image_list.append(img)
        time.sleep(interval_seconds)

    return image_list

# Example usage
url_to_capture = "http://10.12.60.98:8080//shot.jpg"
captured_images = capture_images(url=url_to_capture, interval_seconds=2, num_images=10)

# You can now use the captured_images list for further processing or display


In [1]:
import cv2

def display_images(image_list):
    for i, img in enumerate(image_list):
        cv2.imshow(f'Image {i+1}', img)
        cv2.waitKey(0)  # Wait for any key press to move to the next image

    cv2.destroyAllWindows()

# Example usage
# display_images(captured_images)

In [9]:
import cv2
import os

# Load the face classifier
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def face_cropped(img):
    faces_list = []

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the image
    faces = face_classifier.detectMultiScale(gray, 1.2, 5)

    # Iterate through detected faces
    for (x, y, w, h) in faces:
        # Crop the face from the original image
        cropped_face = img[y: y+h, x:x+w]

        # Resize the face to a common size (e.g., 200x200)
        cropped_face = cv2.resize(cropped_face, (200, 200))

        # Convert the face to grayscale
        cropped_face = cv2.cvtColor(cropped_face, cv2.COLOR_BGR2GRAY)

        # Append the face to the list
        faces_list.append(cropped_face)

    return faces_list

In [10]:
for image in captured_images
faces_list = face_cropped(captured_images[0])

In [22]:
import cv2

def display_images(image_list):
    for i, img in enumerate(image_list):
        cv2.imshow(f'Image {i+1}', img)
        cv2.waitKey(0)  # Wait for any key press to move to the next image

    cv2.destroyAllWindows()

# Example usage
display_images(captured_images)

In [12]:
def face_cropped_from_list(images_list):
    all_faces_list = []

    for img in images_list:
        faces_list = []
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Detect faces in the image
        faces = face_classifier.detectMultiScale(gray, 1.2, 5)

        # Iterate through detected faces
        for (x, y, w, h) in faces:
            # Crop the face from the original image
            cropped_face = img[y: y+h, x:x+w]

            # Resize the face to a common size (e.g., 200x200)
            cropped_face = cv2.resize(cropped_face, (200, 200))

            # Convert the face to grayscale
            cropped_face = cv2.cvtColor(cropped_face, cv2.COLOR_BGR2GRAY)

            # Append the face to the faces_list
            faces_list.append(cropped_face)

        # Append the faces_list to the all_faces_list
        all_faces_list.append(faces_list)

    return all_faces_list

In [13]:
all_faces_list = face_cropped_from_list(captured_images)

In [20]:
len(all_faces_list[9])

1

In [51]:

camera = cv2.VideoCapture(0)  
while True:
 
    ret, frame = camera.read()

    width, height = 64, 64  # Set your desired width and height
    resized_frame = cv2.resize(frame, (width, height))

    gray_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)

    # Expand dimensions and add a channel dimension
    test_image = np.expand_dims(gray_frame, axis=0)
    test_image = np.expand_dims(test_image, axis=3)

    result = classifier.predict(test_image)

    label = ResultMap[np.argmax(result)]

    # Display the result on the frame
    cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Live Camera Feed', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()

cv2.destroyAllWindows()

[['E21CSEU0106', 'E21CSEU0106'], ['E21CSEU0106'], ['E21CSEU0106', 'E21CSEU0106'], ['E21CSEU0106', 'E21CSEU0106'], ['E21CSEU0106'], ['E21CSEU0106'], ['E21CSEU0106', 'E21CSEU0106'], ['E21CSEU0106'], ['E21CSEU0106'], ['E21CSEU0106']]
